In [ ]:
import pandas as pd
import sys
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy as sp
import time
import re
from tqdm import tqdm
import json
from currency_converter import CurrencyConverter
from scipy.stats import ttest_ind
 

In [ ]:
# Merge the simple CSVs based on app_id
gamesdf = pd.read_csv("./games.csv", quotechar='"', escapechar='\\', dtype="str")
genresdf = pd.read_csv("./genres.csv", quotechar='"', escapechar='\\', dtype="str")
gamesdf = pd.merge(gamesdf, genresdf, on="app_id")
reviewsdf = pd.read_csv("./reviews.csv", quotechar='"', escapechar='\\', dtype="str")
gamesdf = pd.merge(gamesdf, reviewsdf, on="app_id")

In [ ]:
### NA values are written as 'N'. Let us fix this below

gamesdf = gamesdf.map(lambda x: np.nan if x == "N" else x)


### The languages column has <strong> tags around * symbols in it sometimes. Let's get rid of these.

gamesdf["languages"] = gamesdf["languages"].str.replace("<strong>*</strong>", "")


### Those asterixes correspond to having full audio support for those languages.
# We don't really care about this so let's get rid of that too
gamesdf["languages"] = gamesdf["languages"].str.replace("<br>languages with full audio support", "")


### Used to check if values are converted correctly. No rows should display if all values with 'N' are properly converted to NaN
# matches = gamesdf == 'N'
# display(gamesdf[matches.any(axis=1)])





In [ ]:
### Some columns aren't typed properly. For example is_free is typed as an object instead of a boolean.

# Let's convert the release date to datetimes
gamesdf["release_date"] = pd.to_datetime(gamesdf["release_date"])

# Now let's convert the is_free column. It uses 0's and 1's to represent false or true respectively
# Let's map that to a boolean
gamesdf["is_free"] = gamesdf["is_free"].apply(lambda x : False if x == '0' else True)


# Let's convert the languages into a list of of strings
gamesdf["languages"] = gamesdf["languages"].str.split(",")




### Let's convert the following columns to integers
toIntegerColumnNames = []

# Let's convert the review scores as well as the positive, negative, and total review counts to integers
toIntegerColumnNames.append("review_score") 
toIntegerColumnNames.append("positive")
toIntegerColumnNames.append("negative")
toIntegerColumnNames.append("total")

# Let's convert the metacritic scores to integers too
toIntegerColumnNames.append("metacritic_score")

# Let's convert the recommendation counts to integers
toIntegerColumnNames.append("recommendations")


# Let's convert the steamspy user scores, score ranks, positive review counts, and negative review counts to integers
toIntegerColumnNames.append("steamspy_user_score")
toIntegerColumnNames.append("steamspy_score_rank")
toIntegerColumnNames.append("steamspy_positive")
toIntegerColumnNames.append("steamspy_negative")


# Okay let's convert all of these to integers now! :)
for columnName in toIntegerColumnNames:
    gamesdf[columnName] = gamesdf[columnName].apply(lambda x: int(x) if pd.notna(x) else x)


# display(gamesdf.head()) #checking work
# print(gamesdf.dtypes) #checking work

In [ ]:
### Unpacking the price_overview column

#Note that I divide these prices by 100 since they don't have appropriate decimals (i.e. 819 = $8.19)
gamesdf["final_price"] = gamesdf["price_overview"].apply(lambda x : (json.loads(x))["final"]/100 if pd.notna(x) else x)
gamesdf["initial_price"] = gamesdf["price_overview"].apply(lambda x : (json.loads(x))["initial"]/100 if pd.notna(x) else x)
gamesdf["currency"] = gamesdf["price_overview"].apply(lambda x : (json.loads(x))["currency"] if pd.notna(x) else x)
gamesdf["discount_percent"] = gamesdf["price_overview"].apply(lambda x : (json.loads(x))["discount_percent"] if pd.notna(x) else x)


print(gamesdf["currency"].unique())
print(gamesdf["discount_percent"].unique())

In [ ]:
### Converting all prices to USD
unique_currencies = gamesdf["currency"].dropna().unique()

conversion_rates = {}

converter = CurrencyConverter(fallback_on_missing_rate=True)
for currency in unique_currencies:
        if currency != 'USD':
            try:
                conversion_rates[currency] = converter.convert(1, currency, 'USD')
            except: #in case anything ever goes wrong with the library
                conversion_rates[currency] = None


#### WARNING: THESE NEXT FEW CURRENCIES ARE MANUALLY BEING UPDATED

## Inputing conversion rates for currencies that don't have one (taken from wise.com)
conversion_rates["SAR"] = 0.2666
conversion_rates["PEN"] = 0.2726
conversion_rates["UAH"] = 0.02433
conversion_rates["COP"] = 0.0002413
conversion_rates["KWD"] = 3.243
conversion_rates["KZT"] = 0.001986
conversion_rates["TWD"] = 0.03011
conversion_rates["AED"] = 0.2723
conversion_rates["VND"] = 0.00003902

for currency, rate in conversion_rates.items():
        currency_rows = (gamesdf['currency'] == currency) #find all rows with this currency
        gamesdf.loc[currency_rows, "currency"] = "USD" #set the currencies to usd now
        gamesdf.loc[currency_rows, "initial_price"] = (gamesdf.loc[currency_rows, "initial_price"] * rate) #convert the initial prices
        gamesdf.loc[currency_rows, "final_price"] = (gamesdf.loc[currency_rows, "final_price"] * rate) #convert the final prices


### Rounding the prices to two decimals
gamesdf["initial_price"] = gamesdf["initial_price"].round(2) #rounding the initial prices
gamesdf["final_price"] = gamesdf["final_price"].round(2) #rounding the final prices

# Fixing free games
gamesdf.loc[gamesdf["is_free"] == True, ["initial_price", "final_price"]] = 0





### TODO: Use some sort of imputation on missing values/figure out what to do with them

display(gamesdf)

In [ ]:
### Now we can begin with simple price analyses
# First create a histrogram with all prices

# Filter out outlier prices
prices = [x for x in gamesdf["initial_price"].dropna() if x < 200]
plt.hist(prices, bins=50)
plt.ylabel("Frequency")
plt.xlabel("Initial game cost ($USD)")
plt.show()

In [ ]:
# Let's get a better look at the main peak of the distribution
prices = [x for x in gamesdf["initial_price"].dropna() if x < 50]
plt.hist(prices, bins=50)
plt.ylabel("Frequency")
plt.xlabel("Initial game cost ($USD)")
plt.show()

In [ ]:
### Now plot games by release date
dates = gamesdf["release_date"].dropna()
plt.hist(dates, bins=50)
plt.ylabel("Frequency")
plt.xlabel("Game release dates")
plt.show()

In [ ]:
### Now let's see if there's a trend between game cost and release date
### Expected: game prices increase over time due to inflation
filteredgamesdf = gamesdf.dropna(subset=["release_date", "initial_price"])
filteredgamesdf = filteredgamesdf[filteredgamesdf["initial_price"] < 100]
filteredgamesdf = filteredgamesdf[filteredgamesdf["initial_price"] != 0] #Uncomment this line to remove free games from the distribution
date_min, date_max = min(filteredgamesdf["release_date"]), max(filteredgamesdf["release_date"])
price_min, price_max = min(filteredgamesdf["initial_price"]), max(filteredgamesdf["initial_price"])
heatmap, x_edges, y_edges = np.histogram2d(filteredgamesdf["release_date"], filteredgamesdf["initial_price"], bins=50)

# Apply log transformation to improve visibility
heatmap_log = np.log1p(heatmap)  # log(1 + x) to handle zeros

#plt.imshow(heatmap.T, origin='lower', cmap='hot', aspect=(x_max / y_max), extent=[x_min, x_max, y_min, y_max])
plt.imshow(heatmap_log.T, origin='lower', cmap='Greens', aspect=(date_max.year / price_max)*5, extent=[date_min, date_max, price_min, price_max])
plt.colorbar(label='Log-Frequency')
plt.xlabel('Release Date')
plt.ylabel('Release Price ($USD)')
plt.title('Game release date vs. Initial Price')

In [ ]:
x = filteredgamesdf["release_date"].map(lambda d: d.toordinal())
y = filteredgamesdf["initial_price"]

# Linear regression
slope, intercept, r_value, p_value, std_err = sp.stats.linregress(x, y)
line = slope * x + intercept  # Predicted values

# Plot
plt.scatter(filteredgamesdf["release_date"], y)
plt.plot(filteredgamesdf["release_date"], line, color='red', label=f"R² = {r_value**2:.3f}")

plt.xlabel("Release Date")
plt.ylabel("Release Price ($USD)")
plt.title(f"Steam game release date vs. initial price")
plt.legend()
plt.show()


### TODO: Add r-squared pearson test to this and line of best fit, this can be our first hypothesis test

### Test 1
Because the correlation coefficient is very low, we see that there is no signifigant influence on a game's price based on the year it was released.
We do see that the general maximum game price increases, particularly from 2008 to 2016, but the average price does not increase.

In [ ]:
# Is there a signficant difference in in review scores between free and paid games?

review_threshold = 50  # Minimum number of reviews to consider

filtered_games = gamesdf[gamesdf["total"] >= review_threshold]

# Get the dfs for review scores for free and paid games
free_games = filtered_games[filtered_games["is_free"] == True]["review_score"].dropna()
paid_games = filtered_games[filtered_games["is_free"] == False]["review_score"].dropna()

# Perform the t-test
t_stat, p_value = ttest_ind(free_games, paid_games, equal_var=False)

# Create a violin plot 
plt.figure(figsize=(8, 6))
sns.violinplot(x=filtered_games["is_free"], y=filtered_games["review_score"])
plt.xlabel("Is Free")
plt.ylabel("Review Score")
plt.title("Distribution of Review Scores for Free vs Paid Games")
plt.xticks(ticks=[0, 1], labels=["Paid", "Free"])
plt.show()

# Output t-test results
print(f"T-statistic: {t_stat}, P-value: {p_value}")

if p_value < 0.05:
    print("The difference in review scores between free and paid games is statistically significant.")
else:
    print("The difference in review scores between free and paid games is not statistically significant.")

### Test 2
For examining the difference in rating scores between free and paid games, we used a 2-sample T-test. Because the P-value we got is 0.08, (above the critical value 0.05), we see no signifigant difference between free and paid game reviews.

In [ ]:
filteredgamesdf["price_overview"] = filteredgamesdf["price_overview"].apply(json.loads)
filteredgamesdf["initial_price_dollars"] = filteredgamesdf["price_overview"].apply(lambda d: d.get("initial", 0) / 100)

# Define price bucket categories in dollars
def price_bucket_dollars(price):
    if price == 0:
        return "Free"
    elif price <= 5:
        return "$0–5"
    elif price <= 15:
        return "$5–15"
    elif price <= 30:
        return "$15–30"
    else:
        return "$30+"

# Apply price buckets
filteredgamesdf["price_bucket"] = filteredgamesdf["initial_price_dollars"].apply(price_bucket_dollars)

# Set price bucket order for plotting
price_order = ["Free", "$0–5", "$5–15", "$15–30", "$30+"]
filteredgamesdf["price_bucket"] = pd.Categorical(
    filteredgamesdf["price_bucket"],
    categories=price_order,
    ordered=True
)

# Create contingency table
contingency = pd.crosstab(filteredgamesdf["genre"], filteredgamesdf["price_bucket"])

# Chi-square test
chi2, p, dof, expected = sp.stats.chi2_contingency(contingency)

# Plot heatmap
sns.heatmap(contingency, annot=True, fmt="d", cmap="Blues")
plt.title("Genre vs. Price Bucket")
plt.xlabel("Price Bucket")
plt.ylabel("Genre")
plt.tight_layout()
plt.show()

# Output results
chi2_test_result = {
    "chi2_statistic": chi2,
    "p_value": p,
    "degrees_of_freedom": dof
}

chi2_test_result



Test 3: This chi squared test shows us that there is a statistically significant relationship between game genre and price tier as teh p value ended up being 0.0 which is 
clearly less than an alpha value of 0.05. This makes sense as in general certain genres of games tend to be priced lower or higher.